In [19]:
#外部ライブラリ
import numpy as np
import h5py
import scipy  
import matplotlib.pyplot as plt
from scipy import signal
import time
from numba import jit

In [37]:
#データ読み込み
hdfpath = r"Obj_1 - recording_20180410_093103-trim-preproc-mc.h5"
with h5py.File(hdfpath,'r') as f:
    # ファイルオブジェクトをイテレートするとファイル直下のオブジェクト名を返す
    data=f['/Object'][0]
    #(1001, 355, 266)

In [79]:
#ガウシアンカーネル
gausian_mask = np.array([
    [1/16,1/8,1/16],
    [1/8,1/4,1/8],
    [1/8,1/4,1/16]
    ])
#畳み込み
def Zet(t):
    return signal.correlate2d(data[t], gausian_mask, mode="same", boundary="wrap") 

#時間を固定し光の強いところを探す

def Pyi(t,index):
    P_Nai = Zet(t)/np.std(data[t])
    if index == "i":
        return P_Nai[np.unravel_index(np.argmax(P_Nai), P_Nai.shape)]
    else:
        return np.unravel_index(np.argmax(P_Nai), P_Nai.shape)
##場所を固定し時間で相関を見る
def ELU(x,y):
    L=20
    col = [np.corrcoef(data[:,x,y],data[:,i,j])  for i in range(-L,L) for j in range(-L,L)]
    cols = [col[k][0][1] for k in range(1600)]
    if (sum(cols)/len(cols)) > 0:
        #print(cols)
        #print(sum(cols)/len(cols))
        return(sum(cols)/len(cols))    
    

In [80]:
#test
Zet(0)
p=Pyi(0,"o")
e=ELU(242,137)
print(p,e)

TypeError: not enough arguments: expected 2, got 1

In [5]:

t = time.time()
[Pyi(t,"o") for t in range(1001)]#5sec
t2 = time.time()
print(t2-t)

4.853693962097168


In [18]:
t3 = time.time()
#355.266
#10.10で17sec
#10.20で45sec
[ELU(i,j) for i in range(10) for j in range(20)]
t4 = time.time()
print(t4-t3)

45.200767993927


In [ ]:
x_star_moto = [ELU(x,y)*Pyi(t,"i") for x in range(256) for y in range(355) for t in range(1000)]
#Sx_star = argmax(x_star_moto)
print("fin")

In [ ]:
#print(x_star_moto)
#x_star を見つけるのが大変